In [34]:
import csv 
import pandas as pd
import requests

from pathlib import Path

In [ ]:
# copy over a VRS-annotated VCF
# TODO: insert Google resource path to VCF here

! gsutil cp <INSERT_VCF_PATH_HERE> $HOME/output.vcf.gz

In [36]:
# write VCF data to csv
!bcftools query -f '%CHROM,%POS,%REF,%ALT,%INFO/VRS_Allele_IDs\n' $HOME/output.vcf.gz > $HOME/vrs_variants.csv

In [37]:
# setup
file_path = f"{Path.home()}/vrs_variants.csv"
variant_dicts = []
i = 0

# read csv
with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)

    for row in csv_reader:
        variant_dict = {}
        # assign the first four entries to their respective keys
        variant_dict['chr'] = row[0]
        variant_dict['pos'] = row[1]
        variant_dict['ref'] = row[2]
        variant_dict['alt'] = row[3]

        # create an array for the remaining VRS IDs
        variant_dict['allele_ids'] = row[4:]
        variant_dicts.append(variant_dict)
        i += 1
        
        # print first three dictionaries
        if i < 4: 
            print(variant_dict)

{'chr': 'chr1', 'pos': '13868', 'ref': 'A', 'alt': 'G', 'allele_ids': ['ga4gh:VA.vDSc19a3c5mQ2mgFZl1WJUZxi4WXrK6W', 'ga4gh:VA.uwLuQlZneuPPZp9heJl75d-qo50g807z']}
{'chr': 'chr1', 'pos': '14464', 'ref': 'A', 'alt': 'T', 'allele_ids': ['ga4gh:VA.Bzxw9XQE_QySENb4CHk0cK43_bjJ9mt4', 'ga4gh:VA.6Sa8k_xvufaLprWriYfycD_YUYKLclvT']}
{'chr': 'chr1', 'pos': '14948', 'ref': 'G', 'alt': 'A', 'allele_ids': ['ga4gh:VA.O2OR4cW8Ok9wanhu9lAo0Q8mR5ZrAUec', 'ga4gh:VA.gSu849zeLlLb7Fxh6jDUxpTz4zX5RRh3']}


In [38]:
# search for metakb study hits for each allele
METAKB_API = "https://dev-search.cancervariants.org/api/v2"

print("MetaKB study matches...")
for d in variant_dicts:
    vrs_ids = d['allele_ids']
    
    for vrs_id in vrs_ids:
        # query metakb studies API
        response = requests.get(f"{METAKB_API}/search/studies?variation={vrs_id}")

        # handle errors
        if response.status_code >= 400:
            print(f"API error: {response.text} ({response.status_code})")
            continue
        
        # output any matching studies
        study_json = response.json()
        studies = study_json["studies"]
        
        if len(studies) > 0:
            print(f"{vrs_id} ({d['chr']}-{d['pos']}-{d['ref']}-{d['alt']})")

            for study in studies:
                print("\t", f"{study['id']} - {study['description']}")

MetaKB study matches...
ga4gh:VA.SOEVGpU16hxYQtJNeRyfq0V-B0rSOGK- (chr1-11796321-G-A)
	 civic.eid:669 - The MTHFR C667T variant was associated with significantly lower relapse-free survival and overall survival in stomach cancer patients treated with 5-Fluorouracil-based therapies. 116 Chinese patients with histologically confirmed gastric cancer were used in this study, and all patients had radical surgery before treatment.
	 civic.eid:1757 - Patients with the wild type (C/C) MTHFR gene are 2.91 times (95% CI: [1.23, 6.89]) more likely to have a positive response to neoadjuvant CRT and  3.25 times more likely not to experience relapse (95% CI: [1.37, 7.72]) than patients with the heterozygous  MTHFR [rs1801133 (C>T)] mutation or  the homzygous (T/T).
